In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import mysql.connector
import os

In [3]:
csv_files = [
    ('orders.csv','orders'),
]

conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='WelcomeToSQL',
    database='elevate_Labs'
)
cursor = conn.cursor()

folder_path = 'C:/Users/singh/OneDrive/Desktop/Elavate-Labs/Elevate-Lab-Task-6'

In [4]:
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

In [5]:
for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)
    df = df.where(pd.notnull(df), None)
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")
    df.columns = [col.replace(' ', '_' ).replace('-', '_' ).replace('.', '_' ) for col in df.columns]
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)
    for _, row in df.iterrows():
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)
    conn.commit()
conn.close()

Processing orders.csv
NaN values before replacement:
Unnamed: 0     0
order_id       0
order_date     0
product_id     0
quantity       0
total_price    0
dtype: int64

